# Evaluate AI agents (Azure AI Agent Service) in Azure AI Foundry

## Objective


This sample demonstrates how to evaluate an AI agent (Azure AI Agent Service) on these important aspects of your agentic workflow:

- Intent Resolution: Measures how well the agent identifies the user’s request, including how well it scopes the user’s intent, asks clarifying questions, and reminds end users of its scope of capabilities.
- Tool Call Accuracy: Evaluates the agent's ability to select the appropriate tools, and process correct parameters from previous steps.
- Task Adherence: Measures how well the agent’s response adheres to its assigned tasks, according to its system message and prior steps.

For AI agents outside of Azure AI Agent Service, you can still provide th agent data in the two formats (either simple data or agent messages) specified in the individual evaluator samples:
- [Intent resolution](https://aka.ms/intentresolution-sample)
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample)
- [Task adherence](https://aka.ms/taskadherence-sample)
- [Response Completeness](https://aka.ms/rescompleteness-sample)



## Time 

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
Creating an agent using Azure AI agent service requires an Azure AI Foundry project and a deployed, supported model. See more details in [Create a new agent](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal).

For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

Important: Make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Prerequisite

Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for AI-assisted evaluators, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name
9) **AGENT_MODEL_DEPLOYMENT_NAME** - The deployment name of the model for your Azure AI agent, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.

### Initializing Project Client

In [11]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet

# Import your custom functions to be used as Tools for the Agent
from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant"

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(toolset=toolset)

### Create an AI agent (Azure AI Agent Service)

In [12]:
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_7aFYY9w0hussRhftmXqUHeL7


### Create Thread

In [13]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_8MLzpIZzQUgxJbUvlI2JBzh2


## Conversation with Agent
Use below cells to have conversation with the agent
- `Create Message[1]`
- `Execute[2]`

### Create Message[1]

In [14]:
# Create message to thread

MESSAGE = "Can you email me weather info for Seattle ?"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_ui3rhUIysNmYWhjxypSvxaJb


### Execute[2]

In [15]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Run finished with status: RunStatus.FAILED
Run failed: {'code': 'invalid_engine_error', 'message': 'No connection matching model: gpt4o'}
Run ID: run_efNOVKtAsa6EbxQtClBDjtZo


### List Messages

In [16]:
for message in project_client.agents.list_messages(thread.id, order="asc").data:
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.USER
Content: Can you email me weather info for Seattle ?
----------------------------------------


# Evaluate

### Get data from agent

In [17]:
from azure.ai.evaluation import AIAgentConverter

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id
file_name = "evaluation_data.jsonl"

# Get a single agent run data
evaluation_data_single_run = converter.convert(thread_id=thread_id, run_id=run_id)

# Run this to save thread data to a JSONL file for evaluation
# Save the agent thread data to a JSONL file
# evaluation_data = converter.prepare_evaluation_data(thread_ids=thread_id, filename=<>)
# print(json.dumps(evaluation_data, indent=4))

### Setting up evaluator

We will select the following evaluators to assess the different aspects relevant for agent quality: 

- [Intent resolution](https://aka.ms/intentresolution-sample): measures the extent of which an agent identifies the correct intent from a user query. Scale: integer 1-5. Higher is better.
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample): evaluates the agent’s ability to select the appropriate tools, and process correct parameters from previous steps. Scale: float 0-1. Higher is better.
- [Task adherence](https://aka.ms/taskadherence-sample): measures the extent of which an agent’s final response adheres to the task based on its system message and a user query. Scale: integer 1-5. Higher is better.


In [18]:
from azure.ai.evaluation import (
    ToolCallAccuracyEvaluator,
    AzureOpenAIModelConfiguration,
    IntentResolutionEvaluator,
    TaskAdherenceEvaluator,
)
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

intent_resolution = IntentResolutionEvaluator(model_config=model_config)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)

task_adherence = TaskAdherenceEvaluator(model_config=model_config)

### Run Evaluator

In [19]:
from azure.ai.evaluation import evaluate

response = evaluate(
    data=file_name,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project=os.environ.get("AIPROJECT_CONNECTION_STRING"),
)
pprint(f'AI Foundary URL: {response.get("studio_url")}')

2025-10-21 09:36:19 -0400   38140 execution.bulk     INFO     Finished 1 / 5 lines.
2025-10-21 09:36:19 -0400   38140 execution.bulk     INFO     Average execution time for completed lines: 0.2 seconds. Estimated time for incomplete lines: 0.8 seconds.
2025-10-21 09:36:20 -0400   35700 execution.bulk     INFO     Finished 1 / 5 lines.
2025-10-21 09:36:20 -0400   35700 execution.bulk     INFO     Average execution time for completed lines: 1.0 seconds. Estimated time for incomplete lines: 4.0 seconds.
2025-10-21 09:36:20 -0400   35700 execution.bulk     INFO     Finished 2 / 5 lines.
2025-10-21 09:36:20 -0400   35700 execution.bulk     INFO     Average execution time for completed lines: 0.54 seconds. Estimated time for incomplete lines: 1.62 seconds.
2025-10-21 09:36:20 -0400   35700 execution.bulk     INFO     Finished 3 / 5 lines.
2025-10-21 09:36:20 -0400   35700 execution.bulk     INFO     Average execution time for completed lines: 0.39 seconds. Estimated time for incomplete lines

======= Run Summary =======

Run name: "tool_call_accuracy_20251021_133619_680181"
Run status: "Completed"
Start time: "2025-10-21 13:36:19.680181+00:00"
Duration: "0:00:05.066712"

======= Combined Run Summary (Per Evaluator) =======

{
    "tool_call_accuracy": {
        "status": "Completed",
        "duration": "0:00:05.066712",
        "completed_lines": 5,
        "failed_lines": 0,
        "log_path": null
    },
    "intent_resolution": {
        "status": "Completed",
        "duration": "0:00:01.998319",
        "completed_lines": 5,
        "failed_lines": 0,
        "log_path": null
    },
    "task_adherence": {
        "status": "Completed",
        "duration": "0:00:02.007719",
        "completed_lines": 5,
        "failed_lines": 0,
        "log_path": null
    }
}


('AI Foundary URL: '
 'https://ai.azure.com/resource/build/evaluation/a7769b5e-2a27-48d3-a06e-b99748e4ada7?wsid=/subscriptions/382e9d43-0c24-4bf6-b807-0a4935bdc6f6/resourceGroups/rg-agashirazi-5194/provider

## Inspect results on Azure AI Foundry

Go to AI Foundry URL for rich Azure AI Foundry data visualization to inspect the evaluation scores and reasoning to quickly identify bugs and issues of your agent to fix and improve.

In [23]:
# alternatively, you can use the following to get the evaluation results in memory

# average scores across all runs
print(response["metrics"])
response

{'rows': [{'inputs.query': [{'role': 'system',
     'content': 'You are a helpful assistant'},
    {'createdAt': '2025-04-04T20:48:01Z',
     'role': 'user',
     'content': [{'type': 'text', 'text': 'How is the weather in London ?'}]}],
   'inputs.response': [{'createdAt': '2025-04-04T20:48:07Z',
     'run_id': 'run_l32Z3SXA5g2bxrs6IXbnRolz',
     'role': 'assistant',
     'content': [{'type': 'tool_call',
       'tool_call_id': 'call_a2XxwMTijdIclDEdZMjEaePv',
       'name': 'fetch_weather',
       'arguments': {'location': 'London'}}]},
    {'createdAt': '2025-04-04T20:48:08Z',
     'run_id': 'run_l32Z3SXA5g2bxrs6IXbnRolz',
     'tool_call_id': 'call_a2XxwMTijdIclDEdZMjEaePv',
     'role': 'tool',
     'content': [{'type': 'tool_result',
       'tool_result': {'weather': 'Cloudy, 18°C'}}]},
    {'createdAt': '2025-04-04T20:48:09Z',
     'run_id': 'run_l32Z3SXA5g2bxrs6IXbnRolz',
     'role': 'assistant',
     'content': [{'type': 'text',
       'text': 'The weather in London is curre